In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Database/Videov2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1hxquepQDQDthiwjAugsHGys5mXviQfcq/Database/Videov2


Model

In [ ]:
import tensorflow as tf

class GestureModel(tf.keras.Model):
    def __init__(self):
        super(GestureModel, self).__init__()
        self.feature_extraction = tf.keras.applications.DenseNet201(include_top=False)
        self.feature_extraction.trainable = False
        self.Flat = tf.keras.layers.Flatten()

    def call(self, img):
        temp = tf.cast(img, tf.float32)
        # print(temp)
        temp = tf.keras.applications.densenet.preprocess_input(temp)
        temp = self.Flat(self.feature_extraction(temp))
        return temp

In [ ]:
import os
import random

import cv2
import numpy as np
import random
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dirsep = '//'
def get_all_file_path(directory, file_extension):
    all_filepath = [] # Đây là mảng lưu lại tất cả các đường dẫn chứa file có đuôi file_extension tại thư mục directory
    for root, dirs, files in os.walk(directory): # Vòng lặp để quét tất cả các file trong thư mục
        for file in files:
            for ext in file_extension:
                if file.endswith(ext): # Xuất hiện bug có kí hiệu phân cách và không có, dòng if để chuẩn hóa đường dẫn
                    if root[len(root) -  1] != dirsep:
                        all_filepath.append((root + dirsep, file, root + dirsep + file))
                    else:
                        all_filepath.append((root, file, root + file))
    return all_filepath

def load_video(path):
    cap = cv2.VideoCapture(path)
    frame_list = []
    while (cap.isOpened()):
        try:
            ret, frame = cap.read()
            frame = cv2.resize(frame, (224, 224))
            frame_list.append(frame)
            if 0xFF == ord('q'):
                break
        except:
            break
    cap.release()
    cv2.destroyAllWindows()
    return frame_list

def optical_flow(prev_frame, frame):
    mask = np.zeros_like(frame)
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculates dense optical flow by Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_gray,
                                        gray,
                                        pyr_scale=0.5,
                                        levels=1,
                                        winsize=5,
                                        iterations=10,
                                        poly_n=5,
                                        poly_sigma=1.1,
                                        flow=cv2.CV_32FC2,
                                        flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)

    # Computes the magnitude and angle of the 2D vectors
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Sets image hue according to the optical flow
    # direction
    mask[..., 0] = angle * 180 / np.pi / 2
    mask[..., 1] = 255

    # Sets image value according to the optical flow
    # magnitude (normalized)
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    # Converts HSV to RGB (BGR) color representation
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    cv2.imshow("dense optical flow", rgb)
    cv2.imshow("normal", frame)
    # cv2.waitKey(30)
    cv2.destroyAllWindows()
    return rgb


all_train_video_path = get_all_file_path('train//', ['.mp4'])
random.shuffle(all_train_video_path)

lst = ['bandaging', 'bowling', 'breakdancing', 'ironing', 'kissing', 'riding_scooter',
           'side_kick', 'tap_dancing', 'texting', 'washing_hair']





In [ ]:
current_odd = 0.0
all_odd = 0.0
model = GestureModel()
xgb = XGBClassifier()
TRAINSAM = 1000
lol = False
dataset_x = []
dataset_y = []

In [ ]:
# for i in range(0, len(all_train_video_path)):
for i in range(0, TRAINSAM):
    try:
    # for t in range(1):
        vid = load_video(all_train_video_path[i][2])
        label = conv_txt_to_label(all_train_video_path[i][0])
        print("LABEL:", label)
        vid = np.asarray(vid)
        print(vid.shape)
        vid_img = []
        vid_opt = []
        for j in range(0, len(vid), 64):
            try:
                # vid_opt.append(optical_flow(vid[j-1], vid[j]))
                vid_img.append(vid[j])
            except:
                break
        vid_img = tf.constant(vid_img, dtype=tf.float32)
        x = model(vid_img)
        for j in range(0, len(x)):
            dataset_x.append(x[j].numpy())
            dataset_y.append(label)
        print(i/TRAINSAM)
    except:
        pass



LABEL: 6
(180, 224, 224, 3)
0.0
LABEL: 7
(300, 224, 224, 3)
0.001
LABEL: 7
(300, 224, 224, 3)
0.002
LABEL: 6
(301, 224, 224, 3)
0.003
LABEL: 8
(299, 224, 224, 3)
0.004
LABEL: 7
(290, 224, 224, 3)
0.005
LABEL: 6
(250, 224, 224, 3)
0.006
LABEL: 0
(240, 224, 224, 3)
0.007
LABEL: 1
(246, 224, 224, 3)
0.008
LABEL: 2
(241, 224, 224, 3)
0.009
LABEL: 8
(301, 224, 224, 3)
0.01
LABEL: 5
(300, 224, 224, 3)
0.011
LABEL: 2
(299, 224, 224, 3)
0.012
LABEL: 0
(300, 224, 224, 3)
0.013
LABEL: 6
(301, 224, 224, 3)
0.014
LABEL: 7
(300, 224, 224, 3)
0.015
LABEL: 7
(250, 224, 224, 3)
0.016
LABEL: 3
(300, 224, 224, 3)
0.017
LABEL: 1
(232, 224, 224, 3)
0.018
LABEL: 2
(250, 224, 224, 3)
0.019
LABEL: 2
(301, 224, 224, 3)
0.02
LABEL: 5
(299, 224, 224, 3)
0.021
LABEL: 8
(250, 224, 224, 3)
0.022
LABEL: 6
(150, 224, 224, 3)
0.023
LABEL: 3
(300, 224, 224, 3)
0.024
LABEL: 5
(299, 224, 224, 3)
0.025
LABEL: 1
(98, 224, 224, 3)
0.026
LABEL: 1
(301, 224, 224, 3)
0.027
LABEL: 5
(301, 224, 224, 3)
0.028
LABEL: 4
(300, 224,

In [ ]:
dataset_x = np.asarray(dataset_x)
dataset_y = np.asarray(dataset_y, dtype=np.int)
# xgb.fit(dataset_x, dataset_y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:
print(dataset_x.shape)

(4423, 94080)


In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(dataset_x, dataset_y)

SVC()

In [ ]:
print(5)
import pickle
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

5


In [ ]:
def conv_txt_to_label(str):
    global lst
    for i in range(0, len(lst)):
        st = 'train//' + lst[i] + '//'
        st_ = 'test//' + lst[i] + '//'
        if str == st:
            return i
        if str == st_:
            return i
    return -1

In [ ]:
def check(i, xgb_model):
    vid = load_video(all_train_video_path[i][2])
    label = conv_txt_to_label(all_train_video_path[i][0])
    print("LABEL:", label)
    vid_img = []
    for j in range(0, len(vid), 16):
        try:
            vid_img.append(vid[j])
        except:
            break
    # print(len(vid_img))
    vid_img = tf.constant(vid_img, dtype=tf.float32)
    # print(vid_img)
    x = model(vid_img)
    dataset_x_eval = []
    for j in range(0, len(x)):
        dataset_x_eval.append(x[j].numpy())
    dataset_x_eval = np.asarray(dataset_x_eval)
    dataset_y_pred = xgb_model.predict(dataset_x_eval)
    cnt = [0 for i in range(10)]
    for j in range(0, len(dataset_y_pred)):
        cnt[dataset_y_pred[j]] += 1
    mx = -1
    mx_idx = -1
    for j in range(0, 10):
        if mx < cnt[j]:
            mx_idx = j
            mx = cnt[j]
    return mx_idx

all_train_video_path = get_all_file_path('test//', ['.mp4'])
CORRECT = 0
INCORRECT = 0
for i in range(0, len(all_train_video_path)):
    try:
        label = conv_txt_to_label(all_train_video_path[i][0])
        k = check(i, clf)
        if label == k:
            CORRECT += 1
        else:
            INCORRECT += 1
        print(CORRECT/(CORRECT+INCORRECT), label, k)
    except:
        pass

LABEL: 2
1.0 2 2
LABEL: 2
1.0 2 2
LABEL: 2
1.0 2 2
LABEL: 2
1.0 2 2
LABEL: 2
1.0 2 2
LABEL: 2
0.8333333333333334 2 8
LABEL: 2
0.8571428571428571 2 2
LABEL: 2
0.875 2 2
LABEL: 2
0.8888888888888888 2 2
LABEL: 2
0.8 2 6
LABEL: 2
0.7272727272727273 2 6
LABEL: 2
0.6666666666666666 2 7
LABEL: 2
0.6153846153846154 2 7
LABEL: 2
0.6428571428571429 2 2
LABEL: 2
0.6 2 7
LABEL: 2
0.5625 2 6
LABEL: 2
0.5294117647058824 2 7
LABEL: 2
0.5555555555555556 2 2
LABEL: 2
0.5789473684210527 2 2
LABEL: 2
0.55 2 6
LABEL: 2
0.5238095238095238 2 7
LABEL: 2
0.5454545454545454 2 2
LABEL: 2
0.5652173913043478 2 2
LABEL: 2
0.5833333333333334 2 2
LABEL: 2
0.6 2 2
LABEL: 2
0.5769230769230769 2 6
LABEL: 2
0.5925925925925926 2 2
LABEL: 2
0.6071428571428571 2 2
LABEL: 2
0.6206896551724138 2 2
LABEL: 2
0.6333333333333333 2 2
LABEL: 2
0.6451612903225806 2 2
LABEL: 2
0.625 2 7
LABEL: 2
0.6363636363636364 2 2
LABEL: 2
0.6470588235294118 2 2
LABEL: 2
0.6285714285714286 2 7
LABEL: 2
0.6111111111111112 2 8
LABEL: 2
0.621621621